<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Clockify - Send time entries database to a Google Sheets spreadsheet
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Clockify/Clockify_Send_time_entries_database_to_Google_Sheets_spreadsheet.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedbacks</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Clockify+-+Send+time+entries+database+to+a+Google+Sheets+spreadsheet:+Error+short+description">Bug report</a>

**Tags:** #clockify #timeentry #database #workspace #user #task #project #gsheet #automation 

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook will send the time entries database from Clockify to a Google Sheets spreadsheet. This is usefull for organizations to keep track of their time entries and analyze them.

**References:**
- [Clockify API Documentation](https://docs.clockify.me/#tag/Time-entry/operation/getTimeEntries)

## Input

### Import libraries

In [ ]:
import requests
import naas
import pandas as pd
from datetime import datetime
from naas_drivers import gsheet

### Setup variables
[Get your Clockify API key](https://clockify.me/user/settings) and set the variables below:

- `api_key`: Clockify API Key
- `workspace_id`: ID of the workspace
- `user_id`: ID of the user to get time entries from
- `start_date`: Start date of the timeframe to create the time entries database
- `end_date`: End date of the timeframe to create the time entries database

Share your Google Sheets spreadsheet with our service account : 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com and set the variables below:

- `spreadsheet_url`: Google Sheets spreadsheet URL
- `sheet_name`: Google Sheets sheet name

In [ ]:
api_key = naas.secret.get("CLOCKIFY_API_KEY") or "YOUR_API_KEY"
workspace_id = "626f9e3b36c2670314c0386e" #"<WORKSPACE_ID>"
start_date = "2023-01-01"
end_date = datetime.now().astimezone().isoformat()#.strftime("%Y-%m-%d")
cron = "15 * * * *"
spreadsheet_url = ""
sheet_name = ""

## Model

### Function: Flatten the nested dict

In [ ]:
# Flatten the nested dict
def flatten_dict(d, parent_key='', sep='_'):
    """
    Flattens a nested dictionary into a single level dictionary.

    Args:
        d (dict): A nested dictionary.
        parent_key (str): Optional string to prefix the keys with.
        sep (str): Optional separator to use between parent_key and child_key.

    Returns:
        dict: A flattened dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

### Function: Get referentials from workspace

In [ ]:
def get_data(api_key, workspace_id, endpoint):
    # Init
    page = 1
    df = pd.DataFrame()
    
    while True:
        # Requests
        url = f"https://api.clockify.me/api/v1/workspaces/{workspace_id}/{endpoint}"
        headers = {
            "X-Api-Key": api_key
        }
        params = {
            "page": page,
            "page-size": 100
        }
        res = requests.get(url, headers=headers, params=params)
        data = res.json()
        if len(data) > 0:
            for d in data:
                res = flatten_dict(d)
                tmp_df = pd.DataFrame([res])
                df = pd.concat([df, tmp_df])
        else:
            break
        page += 1
    return df.reset_index(drop=True)

### Get all users

In [ ]:
df_users = get_data(api_key, workspace_id, "users")
df_users = df_users.rename(columns={"id": "userId", "name": "userName"})
df_users = df_users[["userId", "userName", "email"]]
print("Users fetched:", len(df_users))
df_users.head(1)

### Get time entries

In [ ]:
def get_time_entries(
    api_key,
    workspace_id,
    user_id,
    start_date,
    end_date
):
    # Init
    start_date = datetime.strptime(start_date, "%Y-%m-%d").astimezone().isoformat() # Format date
    page = 1
    df = pd.DataFrame()
    
    # Get raw data
    while True:
        url = f"https://api.clockify.me/api/v1/workspaces/{workspace_id}/user/{user_id}/time-entries"
        headers = {"X-Api-Key": api_key}
        params = {
            "start": start_date,
            "end": end_date,
            "page": page,
            "page-size": 100
        }
        res = requests.get(url, headers=headers, params=params)
        data = res.json()
        if len(data) > 0:
            for d in data:
                res = flatten_dict(d)
                tmp_df = pd.DataFrame([res])
                df = pd.concat([df, tmp_df]).reset_index(drop=True)
        else:
            break
        page += 1
    return df.reset_index(drop=True)

# Init
database = pd.DataFrame()

# Loop
for row in df_users.itertuples():
    user_id = row.userId
    user_name = row.userName
    user_email = row.email
    
    # Get entries
    df_time_entries = get_time_entries(api_key, workspace_id, user_id, start_date, end_date)

    # Concat
    database = pd.concat([database, df_time_entries]).reset_index(drop=True)
    
print("Time entries fetched:", len(database))
database.head(3)

### Get all projects

In [ ]:
df_projects = get_data(api_key, workspace_id, "projects")
df_projects = df_projects.rename(columns={"id": "projectId", "name": "projectName"})
df_projects = df_projects[["projectId", "projectName", "clientId"]]
print("Projects fetched:", len(df_projects))
df_projects.head(1)

### Get all clients

In [ ]:
df_clients = get_data(api_key, workspace_id, "clients")
df_clients = df_clients.rename(columns={"id": "clientId", "name": "clientName"})
df_clients = df_clients[["clientId", "clientName"]]
print("Clients fetched:", len(df_clients))
df_clients.head(1)

### Create final database
Enrich data with referentials from workspace

In [ ]:
final_db = database.copy()

# Final DB
final_db = pd.merge(final_db, df_users, how="left", on="userId")
final_db = pd.merge(final_db, df_projects, how="left", on="projectId")
final_db = pd.merge(final_db, df_clients, how="left", on="clientId")

# Select column
to_select = [
    "id",
    "description",
    "isLocked",
    "timeInterval_start",
    "timeInterval_end",
    "timeInterval_duration",
    "userId",
    "userName",
    "email",
    "projectId",
    "projectName",
    "clientId",
    "clientName",
]
final_db = final_db[to_select]
final_db["Month"] = pd.to_datetime(final_db["timeInterval_end"]).dt.strftime("%Y-%m")
final_db["timeduration_Hours"] = round((pd.to_datetime(final_db["timeInterval_end"]) - pd.to_datetime(final_db["timeInterval_start"])).dt.total_seconds() / 3600, 2)
final_db["Date Extract"] = datetime.now().strftime("%Y%m%d%H%M%S")
print("Time entries fetched:", len(final_db))
final_db.head(1)

## Output

### Send dataframe to Google Sheets

In [ ]:
gsheet.connect(spreadsheet_url).send(sheet_name=sheet_name, data=final_db, append=False)

### Add scheduler

In [ ]:
naas.scheduler.add(cron=cron)

# naas.scheduler.delete()